# Język Python - Wykład 4.

## Context Manager 

    with expression [as (targets)]:
        code block with context of "targets"

* Możliwość utworzenia dynamicznego (w runtime) kontekstu dla wykonania sekcji kodu
* Przykłady użycia: 
   * czytanie z pliku - w kontekście otwarcia i zamknięcia pliku
   * zakładanie i zwalnianie blokad ("lock")
   * zmiana i odtworzenie globalnego stanu
* Obiekt context managera musi implementować protokół: ``__enter__()`` i ``__exit__()``
* [PEP 343](http://www.python.org/dev/peps/pep-0343/)

In [1]:
with open('L4.ipynb') as notebook:
    print(notebook.read(50))

{
 "cells": [
  {
   "cell_type": "markdown",
   "


In [2]:
notebook

<_io.TextIOWrapper name='L4.ipynb' mode='r' encoding='cp1250'>

Konstrukcja:

        with EXPR as VAR:
            BLOCK

tłumaczy się jako:

        mgr = (EXPR)
        exit = type(mgr).__exit__  # Not calling it yet
        value = type(mgr).__enter__(mgr)
        exc = True
        try:
            try:
                VAR = value  # Only if "as VAR" is present
                BLOCK
            except:
                # The exceptional case is handled here
                exc = False
                if not exit(mgr, *sys.exc_info()):
                    raise
                # The exception is swallowed if exit() returns true
        finally:
            # The normal and non-local-goto cases are handled here
            if exc:
                exit(mgr, None, None, None)

In [52]:
class MyContextManager(object):
    
    def __enter__(self):
        print("Context prepared")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("Context closed ", exc_type, exc_value, traceback)

        
with MyContextManager():
    continue
    print("Hello")

SyntaxError: 'continue' not properly in loop (<ipython-input-52-43f8f5680de9>, line 11)

In [4]:
class MyContextManager(object):
    
    def __enter__(self):
        print("Context prepared")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("Context closed ", exc_type, exc_value, traceback)

        
with MyContextManager():
    raise Exception()

Context prepared
Context closed  <class 'Exception'>  <traceback object at 0x00000000054C7948>


Exception: 

In [5]:
class MyContextManager(object):
    
    def __init__(self, custom="Context"):
        self.custom = custom
        
    def __enter__(self):
        print("%s prepared" % self.custom)
    
    def __exit__(self, exc_type, exc_value, traceback):
        print("%s closed " % self.custom, exc_type, exc_value, traceback)

        
with MyContextManager('Dog'):
    print("Hau")

Dog prepared
Hau
Dog closed  None None None


### Context manager z użyciem dekoratora

In [6]:
from contextlib import contextmanager

@contextmanager
def tag(name):
    print("<%s>" % name,)
    yield
    print("</%s>" % name,)

with tag("h1"):
    print("foo",)

print()

with tag("div"):
    print("foo",)



<h1>
foo
</h1>

<div>
foo
</div>


## Metody specjalne (cd..)

### ``__repr__``

In [9]:
class T:
    pass

print([T(), T(), T()])

[<__main__.T object at 0x00000000009F0E10>, <__main__.T object at 0x00000000009F0710>, <__main__.T object at 0x00000000009F0FD0>]


In [8]:
class T:
    def __repr__(self):
        return 'T class'

print([T(), T(), T()])
print(repr(T()))

[T class, T class, T class]
T class


### ``__str__`` i ``__unicode__``

In [10]:
class Car:
    def __init__(self, vendor, model, ccm):
        self.vendor = vendor
        self.model = model
        self.ccm = ccm
        
    def __str__(self):
        return '%s %s (%.1f L)' % (self.vendor, self.model, self.ccm / 1000.0)

print(Car('Porsche', '911', 3600))

Porsche 911 (3.6 L)


### ``__hash__``

In [12]:
class T:
    def __hash__(self):
        return hash("fake hash")

{T(): 1}
        

{<__main__.T at 0x9f0eb8>: 1}

In [13]:
hash("fake hash")

-4763684776134142603

### ``__nonzero__``

In [14]:
class T:
    def __nonzero__(self):
        print(id(self))
        if id(self) / 10 % 2 :
            return True
        else:
            return False
print(bool(T()))        

True


### Emulacja typu kontenera (cd..)

In [15]:
class T():
   
    c = [1, 2, 3]

    def __len__(self):
        return len(self.c)
    
    def __contains__(self, x):
        return x in self.c
    
    def __iter__(self):
        for x in self.c:
            yield x
    
t = T()
print(len(t))
print(2 in t)

3
True


In [16]:
iter(t)

<generator object __iter__ at 0x0000000000A4F558>

In [17]:
list(iter(t))

[1, 2, 3]

## New-style class vs classic classes

In [18]:
class T:
    pass
print(type(T))
print(type(T()))

<class 'type'>
<class '__main__.T'>


In [19]:
class T(object):
    pass
print(type(T))
print(T())

<class 'type'>


* obiekty "starych" klas pythona są realizowane przy użyciu jednego wbudowanego typu "instance"
* obiekty nowych klas, są ni mniej ni więcej definiowanym przez użytkownika nowym typem
* stare klasy występują tylko w Pythonie 2.x jako kompatybilność wsteczna. Python 3 nie posiada ich

In [ ]:
class MyClass(): # old-style

    def __init__(self, param): 
        print('init', param)

    def __new__(cls, param):
        print(cls.__name__, param)
        return object.__new__(cls) 

MyClass(1)

In [ ]:
class MyClass(object): # new-style

    def __init__(self, param): 
        print('init', param)

    def __new__(cls, param):
        print(cls.__name__, param)
        return object.__new__(cls) 

MyClass(1)

## Dziedziczenie, mixiny i MRO

In [20]:
class Car(object):
    
    _TOP_SPEED = None
    
    @property
    def TOP_SPEED(self):
        if self._TOP_SPEED is not None:
            return self._TOP_SPEED
        else:
            raise NotImplementedError('Provide a top speed for a car')
            
    def accelerate(self):
        print('%s accelerated to %d' % (self.__class__.__name__, self.TOP_SPEED))
        
    _color = 'black'
    def color(self):
        print("All cars should be %s" % self._color)
    
    def drive(self):
        self.horn()
        self.accelerate()
    
Car().accelerate()

NotImplementedError: Provide a top speed for a car

In [ ]:
class Fiat(Car):
    _TOP_SPEED = 150

class AlfaRomeo(Car):
    _TOP_SPEED = 250
    
    def accelerate(self, *args, **kwargs):
        print('WROOOOOM ',)
        super(self.__class__, self).accelerate(*args, **kwargs)
    
Fiat().accelerate()
Fiat().color()
AlfaRomeo().accelerate()

In [21]:
class MyCar(object):
    _color = 'red'
    def color(self):
        print("My %s is %s!!!" % (self.__class__.__name__, self._color))
        
class MyFiatWithAlfaRomeoEngine(MyCar, AlfaRomeo, Fiat):
    pass

MyFiatWithAlfaRomeoEngine().accelerate()
MyFiatWithAlfaRomeoEngine().color()


NameError: name 'AlfaRomeo' is not defined

In [22]:
MyFiatWithAlfaRomeoEngine.__mro__

NameError: name 'MyFiatWithAlfaRomeoEngine' is not defined

### Mixin

In [23]:
class HornMixin(object):
    def horn(self):
        print("Beeeep! Beeeep!")
        
class Fiat(HornMixin, Car):
    _TOP_SPEED = 150

class AlfaRomeo(HornMixin, Car):
    _TOP_SPEED = 250
    
Fiat().drive()
AlfaRomeo().drive()        

Beeeep! Beeeep!
Fiat accelerated to 150
Beeeep! Beeeep!
AlfaRomeo accelerated to 250


In [24]:
class OnlyRedCarsCanSpeedingMixin(object):
    
    @property
    def TOP_SPEED(self):
        if self._color is 'red':
            return self._TOP_SPEED
        else:
            return 50

class MyAlfa(OnlyRedCarsCanSpeedingMixin, MyCar, AlfaRomeo):
    pass

class Fiat(OnlyRedCarsCanSpeedingMixin, Fiat):
    pass

MyAlfa().accelerate()
Fiat().accelerate()

MyAlfa accelerated to 250
Fiat accelerated to 50


In [25]:
class Car(object):
    
    TOP_SPEED = 100
           
    def accelerate(self):
        print('%s accelerated to %d' % (
            self.__class__.__name__, self.TOP_SPEED))
    
    def horn(self):
        raise NotImplementedError('Implement how to horn!!!')
    
    def drive(self):
        self.horn()
        self.accelerate()
        
Car().drive()        

NotImplementedError: Implement how to horn!!!

### MRO

In [26]:
O = object
class X(O): pass
class Y(O): pass
class A(X,Y): pass
class B(Y,X): pass
class C(A,B): pass
C.__mro__

TypeError: Cannot create a consistent method resolution
order (MRO) for bases Y, X

In [27]:
O = object
class F(O): pass
class E(O): pass
class D(O): pass
class C(D,F): pass
class B(D,E): pass
class A(B,C): pass
print(A.__mro__) 

(<class '__main__.A'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.D'>, <class '__main__.E'>, <class '__main__.F'>, <class 'object'>)


## flightweight pattern = ``__new__()`` + weakref

* Fabryka singletonów
* Stosujemy wtedy kiedy mamy **wiele obiektów** danego typu ale **niewiele różnych wartości** tych obiektów
* Stan obiektu (jego wartość) powinna być możliwa do wyabstrahowania
* Współdzielenie obiektów w celu zaoszczędzenia zasobów
* Obiekty tworzymy za pomocą fabryki, nowy jeśli nie ma, stary jeśli możemy reużytkować
* Nieprawidłowa implementacja może prowadzić do wycieków pamieci

In [28]:
values = ('2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A')
suits = ('h', 'c', 'd', 's')
 
class Card:
    def __init__(self, value, suit):
        self.value, self.suit = value, suit

    def __repr__(self):
        return "<Card: %s%s>" % (self.value, self.suit)

    def __eq__(self, card):
        return self.value == card.value and self.suit == card.suit

    def __ne__(self, card):
       return not self.__eq__(card)

In [29]:
c1 = Card('J', 'h')
c1

<Card: Jh>

In [31]:
c2 = Card('J', 'h')

In [32]:
c1 == c2

True

In [33]:
id(c1), id(c2)

(10923704, 10848240)

In [34]:
id(c1) == id(c2)

False

In [35]:
import weakref

class FlightweightCard(object):
    _CardPool = weakref.WeakValueDictionary()

    def __new__(cls, value, suit):
        obj = FlightweightCard._CardPool.get(value + suit, None)
        if not obj:
            obj = object.__new__(cls)
            FlightweightCard._CardPool[value + suit] = obj
            obj.value, obj.suit = value, suit

        return obj

In [36]:
c3 = FlightweightCard('J', 'h')
c4 = FlightweightCard('J', 'h')

In [37]:
c3 == c4

True

In [38]:
id(c3) == id(c4)

True

## Wyjątki są też klasami!

In [39]:
5 + '5'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
try:
    5 + '5'
except TypeError:
    print('no-no')

In [40]:
try:
    5 + '5'
except TypeError as e:
    print(type(e), e)

<class 'TypeError'> unsupported operand type(s) for +: 'int' and 'str'


In [41]:
try:
    5 + '5'
except (TypeError, KeyError) as e:
    print(type(e), e)

<class 'TypeError'> unsupported operand type(s) for +: 'int' and 'str'


In [42]:
try:
    dict()[3]
except TypeError as e:
    print(type(e), e)
except KeyError as e:
    print("it's only key error")

it's only key error


In [43]:
class WrongAnswerError(Exception):
    pass

try:
    raise WrongAnswerError('wrong wrong wrong!')
except WrongAnswerError:
    print('good good good')
finally:
    print('bye!')

good good good
bye!


In [44]:
class A(object):
    pass

raise A

TypeError: exceptions must derive from BaseException

In [45]:
import sys, traceback

def lumberjack():
    bright_side_of_death()

def bright_side_of_death():
    return tuple()[0]

try:
    lumberjack()
except IndexError:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print("*** print_tb:")
    traceback.print_tb(exc_traceback, limit=1, file=sys.stdout)
    print("*** print_exception:")
    traceback.print_exception(exc_type, exc_value, exc_traceback,
                              limit=2, file=sys.stdout)
    print("*** print_exc:")
    traceback.print_exc()
    print("*** format_exc, first and last line:")
    formatted_lines = traceback.format_exc().splitlines()
    print(formatted_lines[0])
    print(formatted_lines[-1])
    print("*** format_exception:")
    print(repr(traceback.format_exception(exc_type, exc_value,
                                          exc_traceback)))
    print("*** extract_tb:")
    print(repr(traceback.extract_tb(exc_traceback)))
    print("*** format_tb:")
    print(repr(traceback.format_tb(exc_traceback)))
    print("*** tb_lineno:", exc_traceback.tb_lineno)

*** print_tb:
  File "<ipython-input-45-3e63d7daea82>", line 10, in <module>
    lumberjack()
*** print_exception:
Traceback (most recent call last):
  File "<ipython-input-45-3e63d7daea82>", line 10, in <module>
    lumberjack()
  File "<ipython-input-45-3e63d7daea82>", line 4, in lumberjack
    bright_side_of_death()
IndexError: tuple index out of range
*** print_exc:
*** format_exc, first and last line:
Traceback (most recent call last):
IndexError: tuple index out of range
*** format_exception:
['Traceback (most recent call last):\n', '  File "<ipython-input-45-3e63d7daea82>", line 10, in <module>\n    lumberjack()\n', '  File "<ipython-input-45-3e63d7daea82>", line 4, in lumberjack\n    bright_side_of_death()\n', '  File "<ipython-input-45-3e63d7daea82>", line 7, in bright_side_of_death\n    return tuple()[0]\n', 'IndexError: tuple index out of range\n']
*** extract_tb:
[('<ipython-input-45-3e63d7daea82>', 10, '<module>', 'lumberjack()'), ('<ipython-input-45-3e63d7daea82>', 4, 'lu

Traceback (most recent call last):
  File "<ipython-input-45-3e63d7daea82>", line 10, in <module>
    lumberjack()
  File "<ipython-input-45-3e63d7daea82>", line 4, in lumberjack
    bright_side_of_death()
  File "<ipython-input-45-3e63d7daea82>", line 7, in bright_side_of_death
    return tuple()[0]
IndexError: tuple index out of range


### Wyjątki i dekoratory

In [46]:
class retry(object):
    
    def __init__(self, num):
        self.num = num
    
    def __call__(self, f):
        def wrapped():
            while True:
                try:
                    f()
                except Exception as e:
                    if self.num > 0:
                        print('Retry', self.num)
                        self.num -= 1
                    else:
                        raise e
        return wrapped

@retry(10)
def i_always_fail():
    print('try')
    raise Exception('I always fail')
    
i_always_fail()

try
Retry 10
try
Retry 9
try
Retry 8
try
Retry 7
try
Retry 6
try
Retry 5
try
Retry 4
try
Retry 3
try
Retry 2
try
Retry 1
try


Exception: I always fail